# Lesson 83

In [5]:
#from sklearn.datasets import load_boston
from sklearn.datasets import fetch_openml
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import pandas as pd
import numpy as np


In [ ]:
# Gather Data
#boston_dataset = load_boston()
boston_dataset = fetch_openml(name='boston', version=1)
data = pd.DataFrame(data=boston_dataset.data, columns=boston_dataset.feature_names)
features = data.drop(['INDUS', 'AGE'], axis=1)
#features.dtypes
features['CHAS'] = features['CHAS'].astype('float64')
features['RAD'] = features['RAD'].astype('float64')
#features.dtypes

In [7]:
log_prices = np.log(boston_dataset.target).to_numpy()
# add to_numpy() method to force return a Array otherwise returns a serie
#print(type(log_prices))
target = pd.DataFrame(log_prices, columns=['PRICE'])
#print(target)

In [22]:
CRIME_IDX = 0
ZN_IDX = 1
CHAS_IDX = 2
RM_IDX = 4
PTRATIO_IDX = 8

#property_stats = features.mean().to_numpy().reshape(1,11)
property_stats = features.mean().values.reshape(1, 11)
#property_stats

array([[3.61352356e+00, 1.13636364e+01, 6.91699605e-02, 5.54695059e-01,
        6.28463439e+00, 3.79504269e+00, 9.54940711e+00, 4.08237154e+02,
        1.84555336e+01, 3.56674032e+02, 1.26530632e+01]])

In [11]:
regr = LinearRegression().fit(features, target)
fitted_vals = regr.predict(features)

# Chalenge: calculate MSE and RMSE using scikit-learn
MSE = mean_squared_error(target, fitted_vals )
RMSE = np.sqrt(MSE)
RMSE

np.float64(0.18751213519713034)

In [30]:
def get_log_estimate(nr_rooms, students_per_classroom, 
                     next_to_river=False, 
                     high_confidence=False):
    # Configure Property
    property_stats[0][RM_IDX] = nr_rooms
    property_stats[0][PTRATIO_IDX] = students_per_classroom
    if next_to_river:
        property_stats[0][CHAS_IDX] = 1
    else:
        property_stats[0][CHAS_IDX] = 0

    # Make prediction
    log_estimate = regr.predict(property_stats)[0][0]

    # Calc range
    if high_confidence:
        # calc 2 ST. D
        upper_bound = log_estimate + 2* RMSE
        lower_bound = log_estimate - 2* RMSE
        interval = 95
    else:
        # calc 1 ST. D.
        # calc 2 ST. D
        upper_bound = log_estimate + RMSE
        lower_bound = log_estimate - RMSE
        interval = 68
    
    return log_estimate, upper_bound, lower_bound, interval


In [32]:
get_log_estimate(3,10, next_to_river=False, high_confidence=True)

/mnt/ssd/proj/python/My_IA/020/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


(np.float64(3.045868761493372),
 np.float64(3.4208930318876325),
 np.float64(2.6708444910991114),
 95)